## Using Dask with Numba

In [ ]:
import numpy
from matplotlib import pyplot, cm, cbook
%matplotlib inline

from numba import guvectorize

In [ ]:
import dask.array as da

In [ ]:
@guvectorize(['(int16[:,:], int16[:], int16[:,:])',
              '(int32[:,:], int32[:], int32[:,:])',
              '(float32[:,:], float32[:], float32[:,:])',
              '(float64[:,:], float64[:], float64[:,:])'], '(n,n),(m)->(m,m)')
def restrict_2d_gvec(fine, size, coarse):
    I, J = coarse.shape
    for i in range(1, I - 1):
        for j in range(1, J - 1):
            coarse[i, j] = (
                    1/16 * (
                    fine[2 * i - 1, 2 * j - 1] +
                    fine[2 * i - 1, 2 * j + 1] +
                    fine[2 * i + 1, 2 * j - 1] +
                    fine[2 * i + 1, 2 * j + 1]) +

                    1/8 * (
                    fine[2 * i, 2 * j - 1] +
                    fine[2 * i, 2 * j + 1] +
                    fine[2 * i - 1, 2 * j] +
                    fine[2 * i + 1, 2 * j]) +

                    1/4 * fine[i, j])


In [ ]:
filename = cbook.get_sample_data('jacksboro_fault_dem.npz', asfileobj=False)
with numpy.load(filename) as dem:
    z = dem['elevation']

In [ ]:
z = numpy.float64(z)
z = z[:300, :300]

### Create dask array from `z`

In [ ]:
d = da.from_array(z, chunks=(z.shape[0]//3))

In [ ]:
d.chunks

In [ ]:
#ghost all chunk boundaries by 2 
#because you have to halve them later
g = da.ghost.ghost(d, depth={0: 2, 1: 2}, 
                  boundary={0: 'reflect', 1: 'reflect'})

#function to map onto blocks
def restrict_func(block):
    y = numpy.zeros_like(block[::2,::2])
    return restrict_2d_gvec(block, numpy.empty(y.shape[0]), y)

#map blocks, specify new chunksize = old chunksize / 2
g2 = g.map_blocks(restrict_func, chunks=(g.chunks[0][0]//2, 
                                        g.chunks[0][1]//2))

In [ ]:
a = g2.compute()

In [ ]:
pyplot.figure(figsize=(8, 8))
pyplot.imshow(a, cmap=cm.viridis)

In [ ]:
res = da.ghost.trim_internal(g2, {0: 1, 1: 1})

In [ ]:
a = res.compute()

In [ ]:
res.chunks

In [ ]:
pyplot.figure(figsize=(8, 8))
pyplot.imshow(a, cmap=cm.viridis)

## Large problem

In [ ]:
x = da.random.random(size=(50000, 50000), chunks=(5000, 5000))

In [ ]:
#ghost all chunk boundaries by 2 
#because you have to halve them later
g = da.ghost.ghost(x, depth={0: 2, 1: 2}, 
                  boundary={0: 'reflect', 1: 'reflect'})

#function to map onto blocks
def restrict_func(block):
    y = numpy.zeros_like(block[::2,::2])
    return restrict_2d_gvec(block, numpy.empty(y.shape[0]), y)

#map blocks, specify new chunksize = old chunksize / 2
g2 = g.map_blocks(restrict_func, chunks=g.chunks[0][:2])

#trim off remaining ghosting
res = da.ghost.trim_internal(g2, {0: 1, 1: 1})

In [ ]:
a = res.compute()

In [ ]:
a.shape